# 加载类库

In [38]:
import pandas as pd
import os
import numpy as np
import matplotlib
import math
import operator
import xgboost as xgb
import matplotlib.pyplot as plt
import datetime
from sklearn.model_selection import KFold

In [46]:
user_deal_df = pd.read_csv('../feature/predeal/train_user_deal.csv')
user_deal_df.head()

,uid,age,active_date,limit,age_20,age_25,age_30,age_35,age_40,age_45,...,sex_1,sex_2,limit_5000.0,limit_10000.0,limit_15000.0,limit_20000.0,limit_25000.0,user_time_gap,limit_age,limit_active_time
0,26308,30,2016-02-16,15000.0,0,0,1,0,0,0,...,1,0,0,0,1,0,0,259,500.000000,57.915058
1,78209,40,2016-02-21,5000.0,0,0,0,0,1,0,...,1,0,1,0,0,0,0,254,125.000000,19.685039
2,51930,35,2016-04-19,25000.0,0,0,0,1,0,0,...,1,0,0,0,0,0,1,196,714.285714,127.551020
3,10113,25,2016-03-12,25000.0,0,1,0,0,0,0,...,1,0,0,0,0,0,1,234,1000.000000,106.837607
4,17067,35,2016-02-16,15000.0,0,0,0,1,0,0,...,1,0,0,0,1,0,0,259,428.571429,57.915058


# loan_sum 映射为真实值

In [32]:
def deal_loan_sum(df):
    df['loan_sum_real'] = np.round(5**df['loan_sum']-1,2)
    fields_to_drop = ['loan_sum']
    df = df.drop( fields_to_drop, axis = 1 )
    df = df.rename(columns={'loan_sum_real': 'loan_sum'})
    return df

# t_user处理

In [44]:
def deal_user(user_df, limit_time):

    #购买时间进行处理
#     limit_time = '2016-12-01'
    user_df['limit'] = np.round(5**user_df['limit']-1,2)
    dummy_fields = ['age', 'sex', 'limit']
    for each in dummy_fields:
        dummies = pd.get_dummies( user_df.loc[:, each], prefix=each ) 
        user_df = pd.concat( [user_df, dummies], axis = 1 )
    
    #删除不需要的列
    fields_to_drop = ['sex']
    user_df = user_df.drop( fields_to_drop, axis = 1 )
        
    user_dt=datetime.datetime.strptime(limit_time ,'%Y-%m-%d')
    user_df['user_time_gap']=user_df['active_date'].apply(lambda x: user_dt - (datetime.datetime.strptime(x,'%Y-%m-%d')))
    user_df['user_time_gap']=user_df['user_time_gap'].apply(lambda x: x.days)
#     user_df['user_time_gap']=user_df['user_time_gap'] - user_df['user_time_gap'].min()  # 减去最小值
#     user_df['user_time_gap']=user_df['user_time_gap']/user_df['user_time_gap'].mean()  # 减去最小值

#     bins = [19, 30, 35, 51]
#     df_age = pd.get_dummies(pd.cut(df['age'], bins))
#     df_age.columns = ['user_20_30', 'user_30_35', 'user_35_50']

#     df_result = df.join(df_age)

    user_df['limit_age'] = user_df['limit']/user_df['age']
    user_df['limit_active_time'] = user_df['limit']/user_df['user_time_gap']

#     print user_df.head()
#     df_result.to_csv('./predeal/span_user_deal.csv', index=False)
    return user_df

# t_order处理

In [51]:
def deal_order(df,  limit_time):
    # order_df=pd.read_csv('./data/t_order.csv')
    #去除price=0的
    index=pd.notnull(df['price'])
    df=df.loc[index,:]

    #购买时间进行处理
    order_dt=datetime.datetime.strptime(limit_time,'%Y-%m-%d')
    df['order_time_gap']=df['buy_time'].apply(lambda x: order_dt - (datetime.datetime.strptime(x,'%Y-%m-%d')))
    df['order_time_gap']=df['order_time_gap'].apply(lambda x: x.days)
   
    #转换为真实值
    df['price'] = np.round(5**df['price']-1, 2)
    df['discount'] = np.round(5**df['discount']-1, 2)
    #删除 price*qty<discount的数据
    df = df[df.price*df.qty >= df.discount]
    
    order_group = df.groupby('uid')
    order_feature = []
    cnt=0
    for uid, group in order_group:
        cnt+=1
        if cnt%10000==0:
            print "count total user:","cnt"
        group = group.reset_index()
        order_feature_dict = {}

        #total信息
        order_feature_dict['uid'] = uid
        order_feature_dict['total_order'] = len(group)
        order_feature_dict['total_qty'] = group['qty'].sum()
        order_feature_dict['total_discount'] = group['discount'].sum()
        order_feature_dict['unique_cate_count'] = len(group['cate_id'].unique())

        order_feature_dict['max_qty'] = group['qty'].max()
        order_feature_dict['max_price'] = group['price'].max()
        order_feature_dict['max_discount'] = group['discount'].max()
        order_feature_dict['max_order_time_gap'] = group['order_time_gap'].max()


        order_feature_dict['min_qty'] = group['qty'].min()
        order_feature_dict['min_price'] = group['price'].min()
        order_feature_dict['min_discount'] = group['discount'].min()
        order_feature_dict['min_order_time_gap'] = group['order_time_gap'].min()

        order_feature_dict['mean_qty'] = group['qty'].mean()
        order_feature_dict['mean_price'] = group['price'].mean()
        order_feature_dict['mean_discount'] = group['discount'].mean()
        order_feature_dict['mean_order_time_gap'] = group['order_time_gap'].mean()
        
        order_feature_dict['std_qty'] = group['qty'].std()
        order_feature_dict['std_price'] = group['price'].std()
        order_feature_dict['std_discount'] = group['discount'].std()
        order_feature_dict['std_order_time_gap'] = group['order_time_gap'].std()


        total_price = 0
        for i in range(len(group)):
            total_price += group.loc[i, 'qty'] * group.loc[i, 'price']

        order_feature_dict['total_price'] = total_price
        total_price_mean = 0
        qty_sum = group['qty'].sum()
        if (qty_sum !=0) :
            total_price_mean = total_price / qty_sum

        order_feature_dict['total_price_mean'] = total_price_mean
        
        order_gap=list(group['order_time_gap'])
        for day_diff in [7,14,21,28,35,42,49,56,63,70,90]:
            order_feature_dict['last_'+str(day_diff)+'_day_order_count']=len([i for i in order_gap if i <=day_diff])
            order_feature_dict['last_'+str(day_diff)+'_day_price_amount']=sum([group.loc[order_gap.index(i),'price'] for i in order_gap if i<=day_diff])
            order_feature_dict['last_'+str(day_diff)+'_day_qty_amount']=sum([group.loc[order_gap.index(i),'qty'] for i in order_gap if i<=day_diff])
            order_feature_dict['last_'+str(day_diff)+'_day_discount_amount']=sum([group.loc[order_gap.index(i),'discount'] for i in order_gap if i<=day_diff])
        order_feature.append(order_feature_dict)

    order_feature_df = pd.DataFrame(order_feature)
    # print order_feature_df.head()
    # order_feature_df.to_csv('./data/t_order_deal.csv', index=False)
    return order_feature_df

# t_click处理

In [18]:
def deal_click(df, limit_time_click):

    ckick_dt=datetime.datetime.strptime(limit_time_click,'%Y-%m-%d %H:%M:%S')
    df['click_gap']=df['click_time'].apply(lambda x: ckick_dt- (datetime.datetime.strptime(x,'%Y-%m-%d %H:%M:%S')))
    df['click_gap_days']=df['click_gap'].apply(lambda x: x.days)

    click_group=df.groupby('uid')
    click_feature=[]
    cnt=0
    for uid,group in click_group:
        cnt+=1
        if cnt%10000==0:
            print "count total user:","cnt"
        click_feature_dict={}
        group=group.reset_index()
        click_feature_dict['uid']=uid
        click_feature_dict['click_count_3m']=len(group)

        loan_click_gap_days=list(group['click_gap_days'])
        for day_diff in [7,14,21,28,35,42,49,56,63,70,90]:
            click_feature_dict['last_'+str(day_diff)+'_day_click_count']=len([i for i in loan_click_gap_days if i <=day_diff])
        click_feature.append(click_feature_dict)

    click_feature_df =  pd.DataFrame(click_feature)
    return click_feature_df
    # print loan_feature_df.head()
    # loan_feature_df.to_csv('./data/t_loan_deal.csv', index=False)
    


# t_loan表处理

In [56]:
def deal_loan(loan_df, loan_limit_time):
#     df = pd.read_csv('./data/t_loan.csv')
    loan_dt=datetime.datetime.strptime(loan_limit_time,'%Y-%m-%d %H:%M:%S')
    loan_df['loan_gap']=loan_df['loan_time'].apply(lambda x: loan_dt- (datetime.datetime.strptime(x,'%Y-%m-%d %H:%M:%S')))
    loan_df['loan_gap']=loan_df['loan_gap'].apply(lambda x: x.days)
    
    #amount 映射为真实值
    loan_df['loan_amount'] = np.round(5**loan_df['loan_amount']-1,2)
    loan_df['order_total_amount_limit'] = loan_df['total_price']/loan_df['limit']

    loan_group=loan_df.groupby('uid')
    loan_feature=[]
    cnt=0
    for uid,group in loan_group:
        cnt+=1
        if cnt%10000==0:
            print "count total user:","cnt"
        loan_feature_dict={}
        group=group.reset_index()
        
        limit_value = group['limit'].mean()
        total_price_value = group['total_price'].mean()
        
        loan_feature_dict['uid']=uid
        loan_feature_dict['loan_count_3m']=len(group)
        loan_feature_dict['loan_amout_3m']=group['loan_amount'].sum()
        loan_feature_dict['loan_amout_3m_limit_gap']=group['loan_amount'].sum() - limit_value
        loan_feature_dict['loan_amout_3m_limit_div']=group['loan_amount'].sum()/limit_value
        
        loan_feature_dict['loan_max_3m']=group['loan_amount'].max()
        loan_feature_dict['loan_max_3m_limit_gap']=group['loan_amount'].max() - limit_value
        loan_feature_dict['loan_max_3m_limit_div']=group['loan_amount'].max()/limit_value
        loan_feature_dict['loan_max_3m_order_total_gap']=group['loan_amount'].max() - total_price_value
        loan_feature_dict['loan_max_3m_order_total_div']=group['loan_amount'].max()/total_price_value
        
        loan_feature_dict['loan_min_3m']=group['loan_amount'].min()
        loan_feature_dict['loan_plannum_amout'] = group['plannum'].sum()
        loan_feature_dict['loan_plannum_max'] = group['plannum'].max()
        loan_feature_dict['loan_plannum_min'] = group['plannum'].min()
        loan_feature_dict['loan_plannum_amout_mean'] = group['loan_amount'].sum() / group['plannum'].sum()
        
        loan_feature_dict['loan_plannum_limit_mean'] = limit_value / group['plannum'].sum()
        loan_feature_dict['loan_plannum_limit_min'] = limit_value / group['plannum'].min()
        loan_feature_dict['loan_plannum_limit_max'] = limit_value / group['plannum'].max()
        loan_feature_dict['loan_plannum_order_total_mean'] = total_price_value / group['plannum'].sum()
        loan_feature_dict['loan_plannum_order_total_min'] = total_price_value / group['plannum'].min()
        loan_feature_dict['loan_plannum_order_total_max'] = total_price_value / group['plannum'].max()

        loan_gap=list(group['loan_gap'])
        for day_diff in [7,14,21,28,35,42,49,56,63,70,90]:
            loan_feature_dict['last_'+str(day_diff)+'_day_loan_count']=len([i for i in loan_gap if i <=day_diff])
            loan_feature_dict['last_'+str(day_diff)+'_day_loan_amount']=sum([group.loc[loan_gap.index(i),'loan_amount'] for i in loan_gap if i<=day_diff])
            loan_feature_dict['last_'+str(day_diff)+'_day_loan_amount_limit_div'] = loan_feature_dict['last_'+str(day_diff)+'_day_loan_amount']/limit_value
            loan_feature_dict['last_'+str(day_diff)+'_day_loan_amount_order_div'] = loan_feature_dict['last_'+str(day_diff)+'_day_loan_amount']/total_price_value
            loan_feature_dict['last_'+str(day_diff)+'_day_loan_amount_limit_gap'] = loan_feature_dict['last_'+str(day_diff)+'_day_loan_amount'] - limit_value
            loan_feature_dict['last_'+str(day_diff)+'_day_loan_amount_order_gap'] = loan_feature_dict['last_'+str(day_diff)+'_day_loan_amount'] - total_price_value
        loan_feature.append(loan_feature_dict)

    loan_feature_df =  pd.DataFrame(loan_feature)

    print loan_feature_df.head()
    # loan_feature_df.to_csv('./data/t_loan_deal.csv', index=False)
    return loan_feature_df

# feature_merge 生成中间表

In [62]:
user_df = pd.read_csv('../data/t_user.csv')
# order_df=pd.read_csv('../data/t_order.csv')
# loan_df=pd.read_csv('../data/t_loan.csv')
# loan_sum_df=pd.read_csv('../data/t_loan_sum.csv')
# click_df=pd.read_csv('../data/t_click.csv')


train_limit_time = '2016-11-01'
span_limit_time = '2016-12-01'
train_click_limit_time = '2016-11-01 00:00:00'
span_click_limit_time = '2016-12-01 00:00:00'
#数据集切割时间
train_time = '2016-11-01'
span_time = '2016-09-01'

#loan表处理
# loan_sum_deal = deal_loan_sum(loan_sum_df)
# loan_sum_deal.to_csv('../feature/predeal/loan_sum_deal.csv', index=False)
# print 'OK'


#生成User表的train和span数据，active需要train_limit_time划分
# train_user_df = deal_user(user_df, train_limit_time)
# train_user_df.to_csv('../feature/predeal/train_user_deal.csv', index=False)
span_user_df = deal_user(user_df, span_limit_time)
span_user_df.to_csv('../feature/predeal/span_user_deal.csv', index=False)
print 'OK'


# # order 处理
# train_order_df = order_df[order_df['buy_time'] < train_time]
# span_order_df = order_df[order_df['buy_time'] >= span_time]
# train_order_feature = deal_order(train_order_df, train_limit_time)
# train_order_feature.to_csv('../feature/predeal/train_order_deal.csv', index=False)
# span_order_feature = deal_order(span_order_df, span_limit_time)
# span_order_feature.to_csv('../feature/predeal/span_order_deal.csv', index=False)
# print 'OK'



# click 处理
# train_click_df = click_df[click_df['click_time'] < train_time]
# span_click_df = click_df[click_df['click_time'] >= span_time]
# train_click_feature = deal_click(train_click_df, train_click_limit_time)
# train_click_feature.to_csv('../feature/predeal/train_click_deal.csv', index=False)
# span_loan_feature = deal_click(span_click_df, span_click_limit_time)
# span_loan_feature.to_csv('../feature/predeal/span_click_deal.csv', index=False)
# print 'OK'

    #
# # loan处理
# train_loan_df = loan_df[loan_df['loan_time'] < train_time]
# span_loan_df = loan_df[loan_df['loan_time'] >= span_time]
# train_loan_feature = deal_loan(train_loan_df, train_click_limit_time)
# train_loan_feature.to_csv('../feature/predeal/train_loan_deal.csv', index=False)
# span_loan_feature = deal_loan(span_loan_df, span_click_limit_time)
# span_loan_feature.to_csv('../feature/predeal/span_loan_deal.csv', index=False)
# print 'OK'


OK


# loan生成中间表

In [57]:
# user_df = pd.read_csv('../data/t_user.csv')
# order_df=pd.read_csv('../data/t_order.csv')
loan_df=pd.read_csv('../data/t_loan.csv')
# loan_sum_df=pd.read_csv('../data/t_loan_sum.csv')
# click_df=pd.read_csv('../data/t_click.csv')


train_limit_time = '2016-11-01'
span_limit_time = '2016-12-01'
train_click_limit_time = '2016-11-01 00:00:00'
span_click_limit_time = '2016-12-01 00:00:00'
#数据集切割时间
train_time = '2016-11-01'
span_time = '2016-09-01'

train_user_df = pd.read_csv('../feature/predeal/train_user_deal.csv')
span_user_df = pd.read_csv('../feature/predeal/span_user_deal.csv')
train_order_df = pd.read_csv('../feature/predeal/train_order_deal.csv')
span_order_df = pd.read_csv('../feature/predeal/span_order_deal.csv')

train_user_df_part = train_user_df[['uid','limit']]
span_user_df_part = span_user_df[['uid','limit']]
train_order_df_part = train_order_df[['uid','total_price']]
span_order_df_part = span_order_df[['uid','total_price']]

train_loan_df_all = pd.merge(loan_df, train_user_df_part, on='uid', how='left')
train_loan_df_all = pd.merge(train_loan_df_all, train_order_df_part, on='uid', how='left')
span_loan_df_all = pd.merge(loan_df, span_user_df_part, on='uid', how='left')
span_loan_df_all = pd.merge(span_loan_df_all, span_order_df_part, on='uid', how='left')


# # loan处理
train_loan_df = train_loan_df_all[train_loan_df_all['loan_time'] < train_time]
span_loan_df = span_loan_df_all[span_loan_df_all['loan_time'] >= span_time]
train_loan_feature = deal_loan(train_loan_df, train_click_limit_time)
train_loan_feature.to_csv('../feature/predeal/train_loan_deal.csv', index=False)
span_loan_feature = deal_loan(span_loan_df, span_click_limit_time)
span_loan_feature.to_csv('../feature/predeal/span_loan_deal.csv', index=False)
print 'OK'


/Users/cuiyazhou01/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
/Users/cuiyazhou01/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """
/Users/cuiyazhou01/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats i

count total user: cnt
count total user: cnt
count total user: cnt
   last_14_day_loan_amount  last_14_day_loan_amount_limit_div  \
0                  12000.0                               2.40   
1                      0.0                               0.00   
2                      0.0                               0.00   
3                      0.0                               0.00   
4                    500.0                               0.05   

   last_14_day_loan_amount_limit_gap  last_14_day_loan_amount_order_div  \
0                             7000.0                           1.218658   
1                           -25000.0                           0.000000   
2                           -15000.0                           0.000000   
3                           -25000.0                           0.000000   
4                            -9500.0                           0.048777   

   last_14_day_loan_amount_order_gap  last_14_day_loan_count  \
0                           

# merge——feature 从CSV组合最后特征

In [63]:
def getcsvToAllFeature():
    train_user_df = pd.read_csv('../feature/predeal/train_user_deal.csv')
    span_user_df = pd.read_csv('../feature/predeal/span_user_deal.csv')
    loan_sum_df = pd.read_csv('../feature/predeal/loan_sum_deal.csv')
    # 训练集特征
    train_order_feature = pd.read_csv('../feature/predeal/train_order_deal.csv')
    train_loan_feature = pd.read_csv('../feature/predeal/train_loan_deal.csv')
    train_click_feature = pd.read_csv('../feature/predeal/train_click_deal.csv')

    train_feature = pd.merge(train_user_df, train_order_feature, on='uid', how='left')
    train_feature = pd.merge(train_feature, train_loan_feature, on='uid', how='left')
    train_feature = pd.merge(train_feature, train_click_feature, on='uid', how='left')

    train_label = pd.merge(train_user_df, loan_sum_df, on='uid', how='left')
    train_label.fillna(0, inplace=True)
    train_label = train_label[['uid', 'loan_sum']]
    train_data = pd.merge(train_label, train_feature, on='uid', how='left')
    train_data.drop(['active_date'], axis=1, inplace=True)

    print "here is ... 1"

    # 跨时间特征
    span_order_feature = pd.read_csv('../feature/predeal/span_order_deal.csv')
    span_loan_feature = pd.read_csv('../feature/predeal/span_loan_deal.csv')
    span_click_feature = pd.read_csv('../feature/predeal/span_click_deal.csv')

    span_feature = pd.merge(span_user_df, span_order_feature, on='uid', how='left')
    span_feature = pd.merge(span_feature, span_loan_feature, on='uid', how='left')
    span_feature = pd.merge(span_feature, span_click_feature, on='uid', how='left')
    span_feature.drop(['active_date'], axis=1, inplace=True)
    span_feature.shape
    span_data = span_feature

    train_data.to_csv('../feature/train_data_deal_all.csv', index=False)
    span_data.to_csv('../feature/span_data_deal_all.csv', index=False)

    print "OK!"
 

In [64]:
getcsvToAllFeature()
print 'OK'

here is ... 1
OK!
OK


# 训练结果